In [1]:
# --- Import Necessary Libraries ---
import praw
import pandas as pd
import time
from datetime import datetime, timedelta
import csv  # for quoting

# --- Logging ---
def log(msg):
    print(msg)

def timestamp():
    return datetime.now().strftime('%H:%M:%S')

# --- Reddit API setup ---
reddit = praw.Reddit(
    client_id="0GgjEdXVoxIm0LTkBxIjNA",
    client_secret="NsY4t_AN6o5E4SSki7YkhVee07DCzw",
    user_agent="game-review-bot"
)

# --- Game and keywords ---
game_name = "Assassin's Creed Shadows"
subreddit = "games"

keywords = [
    "assassin's creed shadows"
]

# --- Date range: daily ---
start_date = datetime(2025, 2, 28)
end_date = datetime(2025, 3, 31)
step = timedelta(days=1)

# --- Storage ---
posts_data = []
comments_data = []
deduped_post_ids = set()

# --- Daily scraping loop ---
current_start = start_date
while current_start < end_date:
    current_end = current_start + step
    start_ts = int(current_start.timestamp())
    end_ts = int(current_end.timestamp())

    log(f"[{timestamp()}] 🔍 Scraping for {game_name} in r/{subreddit} from {current_start.date()} to {current_end.date()}")

    try:
        for kw in keywords:
            for post in reddit.subreddit(subreddit).search(kw, sort="new", time_filter="all", limit=100):
                time.sleep(1)

                if not (start_ts <= post.created_utc < end_ts):
                    continue
                if post.id in deduped_post_ids:
                    continue

                full_text = f"{post.title} {post.selftext}".lower()
                title_body_keywords = [k for k in keywords if k in full_text]
                is_strong_match = "assassin's creed shadows" in full_text

                if is_strong_match or len(title_body_keywords) >= 2:
                    deduped_post_ids.add(post.id)
                    log(f"✅ Matched post: {post.title[:80]}...")

                    posts_data.append({
                        "game": game_name,
                        "keyword": kw,
                        "subreddit": subreddit,
                        "post_id": post.id,
                        "title": post.title,
                        "selftext": post.selftext,
                        "score": post.score,
                        "num_comments": post.num_comments,
                        "created_utc": post.created_utc,
                        "author": str(post.author),
                        "matched_keywords_title_body": ", ".join(title_body_keywords)
                    })

                    # --- Save all comments ---
                    try:
                        post.comments.replace_more(limit=None)
                        all_comments = post.comments.list()
                        log(f"    🗨️ {len(all_comments)} comments in post {post.id}")

                        for comment in all_comments[:300]:
                            try:
                                comments_data.append({
                                    "game": game_name,
                                    "keyword": kw,
                                    "subreddit": subreddit,
                                    "post_id": post.id,
                                    "comment_id": comment.id,
                                    "comment_body": comment.body,
                                    "author": str(comment.author) if comment.author else "deleted",
                                    "score": comment.score,
                                    "created_utc": comment.created_utc
                                })
                            except Exception as e:
                                log(f"⚠️ Skipped bad comment in post {post.id}: {e}")
                                continue

                    except Exception as e:
                        log(f"⚠️ Error fetching comments for post {post.id}: {e}")

    except Exception as e:
        log(f"⛔ Error scraping r/{subreddit} on {current_start.date()} → {e}")

    current_start = current_end  # move to next day

# --- Save data to CSV with clean column formatting ---
posts_df = pd.DataFrame(posts_data).drop_duplicates(subset="post_id")
comments_df = pd.DataFrame(comments_data)

posts_df.to_csv("acs_posts.csv", index=False, encoding='utf-8', quoting=csv.QUOTE_MINIMAL)
comments_df.to_csv("acs_comments.csv", index=False, encoding='utf-8', quoting=csv.QUOTE_MINIMAL)

log(f"\n✅ Done! Saved {len(posts_df)} posts and {len(comments_df)} comments.")

[13:39:26] 🔍 Scraping for Assassin's Creed Shadows in r/games from 2025-02-28 to 2025-03-01
✅ Matched post: Assassin's Creed Shadows - Deep Dive Into Progression...
    🗨️ 1 comments in post 1j0csmz
[13:41:07] 🔍 Scraping for Assassin's Creed Shadows in r/games from 2025-03-01 to 2025-03-02
[13:42:49] 🔍 Scraping for Assassin's Creed Shadows in r/games from 2025-03-02 to 2025-03-03
[13:44:30] 🔍 Scraping for Assassin's Creed Shadows in r/games from 2025-03-03 to 2025-03-04
[13:46:11] 🔍 Scraping for Assassin's Creed Shadows in r/games from 2025-03-04 to 2025-03-05
✅ Matched post: Assassin's Creed Shadows - Deep Dive into the Hideout...
    🗨️ 23 comments in post 1j3h9uk
[13:47:52] 🔍 Scraping for Assassin's Creed Shadows in r/games from 2025-03-05 to 2025-03-06
[13:49:33] 🔍 Scraping for Assassin's Creed Shadows in r/games from 2025-03-06 to 2025-03-07
✅ Matched post: Assassin's Creed Shadows - Dual Paths: Naoe & Yasuke Walkthrough Video...
    🗨️ 19 comments in post 1j4z95r
[13:51:15] 🔍 Scr

In [4]:
# --- Save collected data ---
posts_df = pd.DataFrame(posts_data).drop_duplicates(subset="post_id")
comments_df = pd.DataFrame(comments_data)

posts_df.to_csv("acs_games_posts_att2.csv", index=False)
comments_df.to_csv("acs_games_comments_att2.csv", index=False)

log(f"✅ Done! Saved {len(posts_df)} posts and {len(comments_df)} comments.")


NameError: name 'pd' is not defined